In [1]:
from yelpapi import YelpAPI
import os
import pandas as pd
import pickle

In [10]:
data_dir =  os.path.join(os.getcwd(),'Yelp_Data\\')

In [24]:
api_key = 'Gxa0LqhgTU-G3sf9RuA_kt5dHTxgH-m5BNMdM-0TpN56PYRFdEnoj811SGiz9O2-a5TazMI5VpOzzBH91ZMX9p4PJ1K-ALQ0VSnuuL3t4Yt97lrV-3dBdNikmVC3X3Yx'

In [25]:
yelp_api = YelpAPI(api_key)

In [22]:
# this part is going to be a pain. We can loop through this and get 
# results for each area, but there is a limit on the api request per day 
# per 30 days. Maybe just focus on major cities?

businesses_location = 'Santa Barbara, CA'

In [26]:
# we can play around with the limit and offset parameters 
# to control the number of results and what item to start the pull on
search_results = yelp_api.search_query(term='Internet Service Providers',
                                       location=businesses_location,
                                      limit = 50)

In [28]:
business_df = pd.DataFrame.from_dict(search_results['businesses'])

In [36]:
# drop the phone, display_phone, transactions, is_closed, and image_url columns
# we shouldn't need them
unecessary_cols = ['phone', 'display_phone', 'transactions', 'is_closed', 
                   'image_url']


business_df2 = business_df.drop(unecessary_cols,1)

In [37]:
#loop through businesses
business_reviews = dict()
for iBiz, biz_id in enumerate(business_df2.loc[:,'id']):
    business_name = business_df2['name'][iBiz]
    
    #can only get 3 reviews through yelp api
    #BUT...we have the url...which means it should be easy to "not legally" scrape
    business_reviews[business_name] = yelp_api.reviews_query(biz_id)

In [61]:
businessRev_df = pd.DataFrame()

for biz in business_reviews.keys():
    
    # temporary data frame we can use that will be appended to a master one later
    temp_df = pd.DataFrame.from_dict(business_reviews[biz]['reviews'])
    
    temp_df = temp_df.drop('user',1)
    
    # add column for ISP provider
    temp_df.insert(0,'ISP_name',biz)
    
    # add column for business id
    temp_df.insert(1,'business_id',
                   business_df[business_df['name'] == biz]['id'].item())
    
    temp_df.insert(6,'location',
                       str(business_df2[business_df2['name'] == biz]['location'].item()['display_address']))
    
    temp_df = temp_df.rename(columns = {"id":"rev_id"})
    
    businessRev_df = businessRev_df.append(temp_df,ignore_index = True)

In [119]:
# save data
business_df2.to_csv(data_dir+'businesses.csv', index=False)
businessRev_df.to_csv(data_dir+'businesses_reviews.csv', index = False)

In [87]:
from yelpapi import YelpAPI
import pandas as pd

def Yelp_ScrapeISP (api_key,city_names,business_data = None,
                    business_reviews = None):
    data_dir =  os.path.join(os.getcwd(),'Yelp_Data\\')

    yelp_api = YelpAPI(api_key)
    
    all_business_df = pd.DataFrame()
    all_reviews_df = pd.DataFrame()

    for iCity in cities:
        
        # we can play around with the limit and offset parameters 
        # to control the number of results and what item to start the pull on
        search_results = yelp_api.search_query(term = 'Internet Service Provides',
                                               location = iCity, limit = 50)

        business_df = pd.DataFrame.from_dict(search_results['businesses'])


        # drop the phone, display_phone, transactions, is_closed, and image_url columns
        # we shouldn't need them
        unecessary_cols = ['phone', 'display_phone', 'transactions', 'is_closed','image_url']


        business_df2 = business_df.drop(unecessary_cols,1)

        #loop through businesses
        reviews = dict()

        reviews_df = pd.DataFrame()
        for iBiz, biz_id in enumerate(business_df2.loc[:,'id']):
            business_name = business_df2['name'][iBiz]

            #can only get 3 reviews through yelp api
            #BUT...we have the url...which means it should be easy to "not legally" scrape
            reviews[business_name] = yelp_api.reviews_query(biz_id)

            # temporary data frame we can use that will be appended to a master one later
            temp_df = pd.DataFrame.from_dict(reviews[business_name]['reviews'])

            temp_df = temp_df.drop('user',1)

            # add column for ISP provider
            temp_df.insert(0,'ISP_name',business_name)

            # add column for business id
            temp_df.insert(1,'business_id',biz_id)

            # add column for business location
            temp_df.insert(6,'location',
                           str(business_df2[business_df2['id'] == biz_id]['location'].item()['display_address']))


            temp_df = temp_df.rename(columns = {"id":"rev_id"})

            reviews_df = reviews_df.append(temp_df,ignore_index = True)


        all_business_df = all_business_df.append(business_df2, ignore_index=True)
        all_reviews_df = all_reviews_df.append(reviews_df,ignore_index=True)

    # Save data
    # if no previous files, just save the data. if previous files, append
    if business_data == None and business_reviews == None:
        all_business_df.to_csv(data_dir+'businesses.csv', index=False)
        all_reviews_df.to_csv(data_dir+'businesses_reviews.csv', index = False)

    else: #append data to previous loaded files

        prev_business_df = pd.read_csv(data_dir+business_data)

        prev_reviews_df = pd.read_csv(data_dir+business_reviews)

        new_business_df = prev_business_df.append(all_business_df, ignore_index = True)
        new_reviews_df = prev_reviews_df.append(all_reviews_df, ignore_index = True)

        new_business_df.to_csv(data_dir+'businesses.csv', index=False)
        new_reviews_df.to_csv(data_dir+'businesses_reviews.csv', index = False)

#get the api key by creating an account on yelp and then clicking on Create App. Fill out
#form and it will generate a key for you.
api_key = 'Gxa0LqhgTU-G3sf9RuA_kt5dHTxgH-m5BNMdM-0TpN56PYRFdEnoj811SGiz9O2-a5TazMI5VpOzzBH91ZMX9p4PJ1K-ALQ0VSnuuL3t4Yt97lrV-3dBdNikmVC3X3Yx'
cities = ['Santa Barbara, CA','Los Angeles, CA', 'Santa Clarita, CA', 'Walnut, CA']

business_dataFile = 'businesses.csv'
business_reviewsFile = 'businesses_reviews.csv'

In [88]:
Yelp_ScrapeISP(api_key,cities)